# **3124. Find Longest Calls**

**Table: Contacts**
``` console
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| first_name  | varchar |
| last_name   | varchar |
+-------------+---------+
```
id is the primary key (column with unique values) of this table.

id is a foreign key (reference column) to Calls table.

Each row of this table contains id, first_name, and last_name.

**Table: Calls**
``` console
+-------------+------+
| Column Name | Type |
+-------------+------+
| contact_id  | int  |
| type        | enum |
| duration    | int  |
+-------------+------+
```
(contact_id, type, duration) is the primary key (column with unique values) of this table.

type is an ENUM (category) type of ('incoming', 'outgoing').

Each row of this table contains information about calls, comprising of contact_id, type, and duration in seconds.

Write a solution to find the three longest incoming and outgoing calls.

Return the result table ordered by type, duration, and first_name in descending order and duration must be formatted as HH:MM:SS.

The result format is in the following example.

 

**Example 1:**

**Input:**

**Contacts table:**
``` console
+----+------------+-----------+
| id | first_name | last_name |
+----+------------+-----------+
| 1  | John       | Doe       |
| 2  | Jane       | Smith     |
| 3  | Alice      | Johnson   |
| 4  | Michael    | Brown     |
| 5  | Emily      | Davis     |
+----+------------+-----------+   
```     
**Calls table:**
``` console
+------------+----------+----------+
| contact_id | type     | duration |
+------------+----------+----------+
| 1          | incoming | 120      |
| 1          | outgoing | 180      |
| 2          | incoming | 300      |
| 2          | outgoing | 240      |
| 3          | incoming | 150      |
| 3          | outgoing | 360      |
| 4          | incoming | 420      |
| 4          | outgoing | 200      |
| 5          | incoming | 180      |
| 5          | outgoing | 280      |
+------------+----------+----------+
```      
**Output:**
``` console
+-----------+----------+-------------------+
| first_name| type     | duration_formatted|
+-----------+----------+-------------------+
| Alice     | outgoing | 00:06:00          |
| Emily     | outgoing | 00:04:40          |
| Jane      | outgoing | 00:04:00          |
| Michael   | incoming | 00:07:00          |
| Jane      | incoming | 00:05:00          |
| Emily     | incoming | 00:03:00          |
+-----------+----------+-------------------+
```      
**Explanation:**

- Alice had an outgoing call lasting 6 minutes.
- Emily had an outgoing call lasting 4 minutes and 40 seconds.
- Jane had an outgoing call lasting 4 minutes.
- Michael had an incoming call lasting 7 minutes.
- Jane had an incoming call lasting 5 minutes.
- Emily had an incoming call lasting 3 minutes.
- Note: Output table is sorted by type, duration, and first_name in descending order.

**Solution-1:**

``` sql
# Write your MySQL query statement below

with cte AS(
    SELECT *, RANK() OVER(PARTITION BY type ORDER BY duration DESC) as rnk
    FROM Calls
)
SELECT first_name, type,
  CONCAT(
        LPAD(FLOOR(duration / 3600), 2, '0'), ':',
        LPAD(FLOOR((duration % 3600) / 60), 2, '0'), ':',
        LPAD(duration % 60, 2, '0')
    ) AS duration_formatted
FROM Contacts
JOIN cte 
ON id = contact_id and rnk<=3
ORDER BY  type DESC,duration_formatted DESC, first_name DESC

**Solution-2:**

``` sql
with cte as(
select c1.contact_id,c1.type,c1.duration,c2.first_name,
dense_rank() over(partition by type order by duration desc) as rnk
from calls c1 
left join contacts c2 
on c1.contact_id = c2.id)

select first_name,type,date_format(sec_to_time(duration),'%H:%i:%S') as duration_formatted
from cte 
where rnk <=3 
order by type desc,duration desc,first_name